In [1]:
# Colab-ready BFS maze visualizer
# Paste this entire block into one Colab cell and run.

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import animation
from collections import deque
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
import random

# ---------- parameters ----------
ROWS = 41   # odd numbers give nicer maze generation
COLS = 71   # odd numbers give nicer maze generation
WALL = 1
PATH = 0
START = (1,1)
END = (ROWS-2, COLS-2)
FPS = 30

# ---------- Maze generation (Recursive backtracker) ----------
def generate_maze(rows=ROWS, cols=COLS, seed=None):
    if seed is not None:
        random.seed(seed)
    # Start with all walls
    maze = np.ones((rows, cols), dtype=int)
    # carve passages in odd cells
    def neighbors(r, c):
        for dr, dc in ((0,2),(0,-2),(2,0),(-2,0)):
            nr, nc = r+dr, c+dc
            if 0 < nr < rows-1 and 0 < nc < cols-1:
                yield nr, nc

    stack = []
    r, c = START
    maze[r,c] = PATH
    stack.append((r,c))
    while stack:
        r,c = stack[-1]
        nbrs = [ (nr,nc) for nr,nc in neighbors(r,c) if maze[nr,nc] == WALL ]
        if nbrs:
            nr,nc = random.choice(nbrs)
            # knock down wall between
            wr, wc = (r+nr)//2, (c+nc)//2
            maze[wr,wc] = PATH
            maze[nr,nc] = PATH
            stack.append((nr,nc))
        else:
            stack.pop()
    # ensure start and end are path
    maze[START] = PATH
    maze[END] = PATH
    return maze

# ---------- BFS that yields frames ----------
def bfs_steps(maze, start=START, end=END):
    rows, cols = maze.shape
    visited = [[False]*cols for _ in range(rows)]
    parent = {}
    q = deque()
    q.append(start)
    visited[start[0]][start[1]] = True

    # yield initial state
    yield {'type':'init', 'visited':[], 'frontier':[start], 'parent':None}

    while q:
        cur = q.popleft()
        r,c = cur
        # visiting node
        yield {'type':'visit', 'pos':cur}
        if cur == end:
            # reconstruct path
            path = []
            node = end
            while node != start:
                path.append(node)
                node = parent[node]
            path.append(start)
            path.reverse()
            yield {'type':'path', 'path':path}
            return
        for dr,dc in ((1,0),(-1,0),(0,1),(0,-1)):
            nr, nc = r+dr, c+dc
            if 0 <= nr < rows and 0 <= nc < cols and not visited[nr][nc] and maze[nr,nc] == PATH:
                visited[nr][nc] = True
                parent[(nr,nc)] = (r,c)
                q.append((nr,nc))
                # frontier push
                yield {'type':'frontier', 'pos':(nr,nc)}
    # no path found
    yield {'type':'no_path'}

# ---------- Visualization helpers ----------
cmap = colors.ListedColormap(['white','black','green','red','yellow','lightblue'])
# mapping: 0=path white, 1=wall black, 2=start green, 3=end red, 4=visited yellow, 5=frontier lightblue
bounds=[0,1,2,3,4,5,6]
norm = colors.BoundaryNorm(bounds, cmap.N)

def make_display_grid(maze, visited_set=None, frontier_set=None, path=None):
    grid = np.copy(maze)
    # mark start/end differently by setting values >1 (they don't exist in maze)
    grid = grid.astype(int)
    if visited_set:
        for (r,c) in visited_set:
            if grid[r,c] == PATH:
                grid[r,c] = 4
    if frontier_set:
        for (r,c) in frontier_set:
            if grid[r,c] == PATH or grid[r,c] == 4:
                grid[r,c] = 5
    if path:
        for (r,c) in path:
            grid[r,c] = 4  # final path colored same as visited but will be shown differently by thickness
    # place start/end markers
    sr, sc = START
    er, ec = END
    grid[sr,sc] = 2
    grid[er,ec] = 3
    return grid

# ---------- Animation function generator ----------
def build_animation(maze):
    steps = list(bfs_steps(maze))
    visited = set()
    frontier = set()
    path = None

    fig, ax = plt.subplots(figsize=(12,7))
    ax.set_title('BFS Maze Visualizer', fontsize=16)
    ax.set_xticks([])
    ax.set_yticks([])
    img = ax.imshow(make_display_grid(maze), cmap=cmap, norm=norm)

    visited_coords = []
    frontier_coords = []

    # build a list of frames from the generator steps
    frames = []
    for ev in steps:
        if ev['type'] == 'init':
            frames.append(('draw', None))
        elif ev['type'] == 'frontier':
            frontier_coords.append(ev['pos'])
            frames.append(('frontier', ev['pos']))
        elif ev['type'] == 'visit':
            visited.add(ev['pos'])
            # when visiting, remove from frontier if present
            frontier.discard(ev['pos'])
            frames.append(('visit', ev['pos']))
        elif ev['type'] == 'path':
            path = ev['path']
            frames.append(('path', path))
            break
        elif ev['type'] == 'no_path':
            frames.append(('no_path', None))
            break

    # Animation update
    def update(i):
        nonlocal visited, frontier, path
        typ, payload = frames[i]
        if typ == 'draw':
            grid = make_display_grid(maze, visited, frontier, path)
            img.set_data(grid)
            ax.set_title('BFS start', fontsize=16)
        elif typ == 'frontier':
            frontier.add(payload)
            grid = make_display_grid(maze, visited, frontier, path)
            img.set_data(grid)
            ax.set_title('Adding to frontier: {}'.format(payload), fontsize=12)
        elif typ == 'visit':
            visited.add(payload)
            if payload in frontier:
                frontier.remove(payload)
            grid = make_display_grid(maze, visited, frontier, path)
            img.set_data(grid)
            ax.set_title('Visiting: {}'.format(payload), fontsize=12)
        elif typ == 'path':
            path = payload
            grid = make_display_grid(maze, visited, frontier, path)
            img.set_data(grid)
            # draw path overlay in red line
            path_y = [p[0] for p in path]
            path_x = [p[1] for p in path]
            ax.plot(path_x, path_y, linewidth=3, color='orange')
            ax.set_title('Path found! length={}'.format(len(path)-1), fontsize=14)
        elif typ == 'no_path':
            grid = make_display_grid(maze, visited, frontier, path)
            img.set_data(grid)
            ax.set_title('No path found', fontsize=14)
        return (img,)

    anim = animation.FuncAnimation(fig, update, frames=len(frames), interval=1000//FPS, blit=False)
    plt.close(fig)
    return anim

# ---------- UI and control ----------
maze = generate_maze()

out = widgets.Output(layout=widgets.Layout(border='1px solid black'))
generate_btn = widgets.Button(description='Generate Maze', button_style='info')
solve_btn = widgets.Button(description='Solve (BFS)', button_style='success')
seed_box = widgets.IntText(description='Seed', value=0)
size_r = widgets.IntSlider(description='Rows', min=21, max=81, step=2, value=ROWS)
size_c = widgets.IntSlider(description='Cols', min=21, max=141, step=2, value=COLS)

def show_maze(maze_to_show):
    with out:
        clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(12,7))
        ax.set_xticks([])
        ax.set_yticks([])
        img = ax.imshow(make_display_grid(maze_to_show), cmap=cmap, norm=norm)
        ax.set_title('Maze (start green, end red)', fontsize=14)
        # legend
        legend_elements = [
            plt.Line2D([0],[0], marker='s', color='w', markerfacecolor='green', markersize=10, label='Start'),
            plt.Line2D([0],[0], marker='s', color='w', markerfacecolor='red', markersize=10, label='End'),
            plt.Line2D([0],[0], marker='s', color='w', markerfacecolor='lightblue', markersize=10, label='Frontier'),
            plt.Line2D([0],[0], marker='s', color='w', markerfacecolor='yellow', markersize=10, label='Visited'),
            plt.Line2D([0],[0], marker='s', color='black', markerfacecolor='black', markersize=10, label='Wall')
        ]
        ax.legend(handles=legend_elements, loc='upper right')
        plt.show()

def on_generate(b):
    global maze, START, END
    r = size_r.value
    c = size_c.value
    s = seed_box.value if seed_box.value != 0 else None
    maze = generate_maze(rows=r, cols=c, seed=s)
    # update start & end based on new size
    START = (1,1)
    END = (r-2, c-2)
    show_maze(maze)

def on_solve(b):
    with out:
        clear_output(wait=True)
        anim = build_animation(maze)
        display(HTML(anim.to_jshtml()))

generate_btn.on_click(on_generate)
solve_btn.on_click(on_solve)

controls = widgets.HBox([generate_btn, solve_btn, seed_box])
size_controls = widgets.HBox([size_r, size_c])
display(widgets.VBox([controls, size_controls, out]))

# show initial maze
show_maze(maze)
